## Autoencoder(Combined with CNN?)

In [1]:
import manga109api
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage
%matplotlib inline

In [2]:
manga109_root = "../datasets/Manga109/Manga109_released_2021_12_30"
dataset = manga109api.Parser(manga109_root)

In [3]:
def data(is_train = True):
    count = -1
    for book in dataset.books:
        for page in dataset.get_annotation(book)["page"]:
            count += 1
            if count % 5 == 0 and is_train: continue
            if count % 5 != 0 and (not is_train): continue
            image = tf.convert_to_tensor(skimage.io.imread("../datasets/Manga109/Manga109_released_2021_12_30/padded_text_stripped_images/" + book + "/%03d.jpg" % (page["@index"]))[np.newaxis])
            yield image, image

In [4]:
train_data = tf.data.Dataset.from_generator(data,
                               output_signature=(
                                   tf.TensorSpec(shape=(None,2048,2048,3)),
                                   tf.TensorSpec(shape=(None,2048,2048,3))
                               ))
test_data = tf.data.Dataset.from_generator(lambda: data(False),
                               output_signature=(
                                   tf.TensorSpec(shape=(None,2048,2048,3)),
                                   tf.TensorSpec(shape=(None,2048,2048,3))
                               ))

In [5]:
_input = layers.Input(shape=(2048, 2048, 3))

# Encoder
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(_input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
for i in range(6):
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)
encoded = layers.Flatten()(x)

# Decoder
x = layers.Reshape((16, 16, 32))(encoded)
for i in range(7):
    x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
decoded = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(_input, decoded)
autoencoder.compile(optimizer="adam", loss="mean_squared_error")
encoder = Model(_input, encoded)
decoder = Model(encoded, decoded)

In [ ]:
autoencoder.fit(
    x=train_data,
    epochs=10,
    shuffle=True,
    validation_data=test_data
)

Epoch 1/10
    885/Unknown - 643s 704ms/step - loss: 21984.5566

In [ ]:
autoencoder.save("model/autoencoder.h5")
encoder.save("model/encoder.h5")
decoder.save("model/decoder.h5")

## References

[https://keras.io/examples/vision/autoencoder/](https://keras.io/examples/vision/autoencoder/)